In [1]:
from GPR import GaussianProcessRegressionLengthscale
from GPR_LS_V import GaussianProcessRegressionLengthscaleVariance
from GPR_CHOL_LS_V import GaussianProcessRegressionLengthscaleVarianceCholesky
from GPR_CHOL_LS import GaussianProcessRegressionLengthscaleCholesky
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as pl


np.random.seed(100) 

In [10]:
adt = pd.read_csv("C:/Users/flea0265\Github/DARE-parker-challenge/engineered_data/assay_desurveyed_transformed_Mlith_domain.csv")
domained = pd.read_csv("../../DARE-parker-challenge/engineered_data/blockmodel_domained_NN.csv")


In [11]:
columns_to_replace = adt.columns[17:31]
adt[columns_to_replace] = adt[columns_to_replace].replace(0, np.nan)

In [12]:
mineral = 'Cu_pct'
adt_Cu= adt.loc[:,["HOLEID", 'EAST_mid_tf', 'NORTH_mid_tf', 'RL_mid_tf', mineral]]

adt_Cu = adt_Cu.dropna(subset=[mineral])

In [13]:
counts = adt['HOLEID'].value_counts()
mask = adt['HOLEID'].isin(counts[counts > 40].index) & ~adt['Cu_pct'].isna()
result = adt[mask]
adt_Cu2 = adt_Cu

### 1. Scikit GP

In [ ]:
np.random.seed(100) 
means = [] 
stds = []
lengthscales = []
holeids = []
xs = []
ys = []
x_trains = []
x_tests = []
y_trains = []
y_tests = []
convergeds=[]
sigma_ns = []

for hole in result['HOLEID'].unique():
    # subset assay data by domains
    adt_Cu = adt_Cu2.loc[adt_Cu2['HOLEID'] == hole]
    x = adt_Cu[['EAST_mid_tf','NORTH_mid_tf', 'RL_mid_tf']].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    y = adt_Cu[mineral].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    x = x.reshape(len(adt_Cu),3)
    x = x[:,2].reshape(-1,1)

    random_indices = np.random.permutation(x.shape[0])
    train_ratio = .7
    n_train = int(np.floor(x.shape[0]*train_ratio))
    n_test = x.shape[0]-n_train
    train_idx, test_idx = random_indices[:n_train], random_indices[n_train:]
    train_idx.sort()
    test_idx.sort()
    x_train, x_test = x[train_idx,:], x[test_idx,:]
    y_train, y_test = y[train_idx,:], y[test_idx,:]

    x_test2 = np.linspace(x_test.min(), x_test.max(), 100).reshape(-1, 1)
    signal_variance = 1 #0.15
    lengthscale = 1000000000000 #50000
    sigma_n = 1 #0.1
    kernel = RBF(length_scale=1000000000000) 
    gp = GaussianProcessRegressor(kernel=kernel, alpha=1) 
    gp.fit(x_train, y_train)

    # Predict the mean and standard deviation of the output for the new data
    f_mean_2,f_std_2 = gp.predict(x_test2, return_std=True)



    xs.append(x)
    ys.append(x)
    x_trains.append(x_train)
    x_tests.append(x_test)
    y_trains.append(y_train)
    y_tests.append(y_tests)

    means.append(f_mean_2)
    stds.append(f_std_2)
    holeids.append(hole)

    pl.figure(figsize=(13, 5))
    pl.scatter(x_train, y_train, s=20, color ='red')
    pl.scatter(x_test, y_test,marker="x", s=20)
    pl.plot(x_test2, f_mean_2)
    pl.fill_between(x_test2.ravel(), (f_mean_2-2*f_std_2).ravel(), (f_mean_2+2*f_std_2).ravel(), \
                    facecolor='blue', interpolate=True, alpha=0.1, label='±2*sigma of prediction')
    #sns.lineplot(x=x_test.reshape(-1), y=f_mean_2.reshape(-1),color ='g')

    #sns.lineplot(x=x_test.reshape(-1), y=(f_mean_2-2*f_std_2).reshape(-1),color ='black')
    #sns.lineplot(x=x_test.reshape(-1), y=(f_mean_2+2*f_std_2).reshape(-1),color ='black')


    #sns.lineplot(x=x_test.reshape(-1), y=f_mean_2.reshape(-1),color ='g')
    pl.title('GP prediction of Cu concentrations vs depth'+hole, fontsize=14)

    #pl.xlim([1500,1800])
    #pl.ylim([0,9])
    pl.xlabel('x [normalised]')
    pl.ylabel('y [normalised and zero mean]')
    pl.grid()

### 2. GP optimising lengthscale 

In [ ]:
np.random.seed(100) 
means = [] 
stds = []
lengthscales = []
holeids = []
xs = []
ys = []
x_trains = []
x_tests = []
y_trains = []
y_tests = []
convergeds=[]
sigma_ns = []

for hole in result['HOLEID'].unique():
    # subset assay data by domains
    adt_Cu = adt_Cu2.loc[adt_Cu2['HOLEID'] == hole]
    x = adt_Cu[['EAST_mid_tf','NORTH_mid_tf', 'RL_mid_tf']].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    y = adt_Cu[mineral].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    x = x.reshape(len(adt_Cu),3)
    x = x[:,2].reshape(-1,1)

    random_indices = np.random.permutation(x.shape[0])
    train_ratio = .7
    n_train = int(np.floor(x.shape[0]*train_ratio))
    n_test = x.shape[0]-n_train
    train_idx, test_idx = random_indices[:n_train], random_indices[n_train:]
    train_idx.sort()
    test_idx.sort()
    x_train, x_test = x[train_idx,:], x[test_idx,:]
    y_train, y_test = y[train_idx,:], y[test_idx,:]

    signal_variance = 1 #0.15
    lengthscale = 1000000 #50000
    sigma_n = 1 #0.1
    #gp = GPR("Matern", x_train, y_train, [signal_variance,lengthscale], sigma_n, nu = 5/2)
    x_test2 = np.linspace(x_test.min(), x_test.max(), 100).reshape(-1, 1)

    gp = GaussianProcessRegressionLengthscale("Squared Exponential", x_train, y_train, [signal_variance,lengthscale], sigma_n)
    gp.fit()
    [f_mean_2,f_std_2] = gp.predict(x_test2)

    convergeds.append(gp.res.success)
    xs.append(x)
    ys.append(x)
    x_trains.append(x_train)
    x_tests.append(x_test)
    y_trains.append(y_train)
    y_tests.append(y_tests)

    means.append(f_mean_2)
    stds.append(f_std_2)
    lengthscales.append(gp.hyper_params[1])
    holeids.append(hole)
    sigma_ns.append(gp.sigma_n)

    pl.figure(figsize=(13, 5))
    pl.scatter(x_train, y_train, s=20, color ='red', label='Training points')
    pl.scatter(x_test, y_test,marker="x", s=20, label='Test points')
    pl.plot(x_test2, f_mean_2, label='Mean prediction')
    pl.fill_between(x_test2.ravel(), (f_mean_2-2*f_std_2).ravel(), (f_mean_2+2*f_std_2).ravel(), \
                    facecolor='blue', interpolate=True, alpha=0.1, label='±2*sigma of prediction')
    #sns.lineplot(x=x_test.reshape(-1), y=f_mean_2.reshape(-1),color ='g')

    #sns.lineplot(x=x_test.reshape(-1), y=(f_mean_2-2*f_std_2).reshape(-1),color ='black')
    #sns.lineplot(x=x_test.reshape(-1), y=(f_mean_2+2*f_std_2).reshape(-1),color ='black')


    #sns.lineplot(x=x_test.reshape(-1), y=f_mean_2.reshape(-1),color ='g')
    pl.title('GP prediction of Cu concentrations vs depth'+hole, fontsize=14)

    #pl.xlim([1500,1800])
    #pl.ylim([0,9])
    pl.xlabel('x [normalised]')
    pl.ylabel('y [normalised and zero mean]')
    pl.grid()

### 3. GP optimising lengthscale + variance

In [ ]:
np.random.seed(100) 
means = [] 
stds = []
lengthscales = []
holeids = []
xs = []
ys = []
x_trains = []
x_tests = []
y_trains = []
y_tests = []
convergeds=[]
sigma_ns = []

for hole in result['HOLEID'].unique():
    # subset assay data by domains
    adt_Cu = adt_Cu2.loc[adt_Cu2['HOLEID'] == hole]
    x = adt_Cu[['EAST_mid_tf','NORTH_mid_tf', 'RL_mid_tf']].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    y = adt_Cu[mineral].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    x = x.reshape(len(adt_Cu),3)
    x = x[:,2].reshape(-1,1)

    random_indices = np.random.permutation(x.shape[0])
    train_ratio = .7
    n_train = int(np.floor(x.shape[0]*train_ratio))
    n_test = x.shape[0]-n_train
    train_idx, test_idx = random_indices[:n_train], random_indices[n_train:]
    train_idx.sort()
    test_idx.sort()
    x_train, x_test = x[train_idx,:], x[test_idx,:]
    y_train, y_test = y[train_idx,:], y[test_idx,:]

    signal_variance = 1 #0.15
    lengthscale = 1000000 #50000
    sigma_n = 1 #0.1
    #gp = GPR("Matern", x_train, y_train, [signal_variance,lengthscale], sigma_n, nu = 5/2)
    x_test2 = np.linspace(x_test.min(), x_test.max(), 100).reshape(-1, 1)

    gp = GaussianProcessRegressionLengthscaleVariance("Squared Exponential", x_train, y_train, [signal_variance,lengthscale], sigma_n)
    gp.fit()
    [f_mean_2,f_std_2] = gp.predict(x_test2)

    convergeds.append(gp.res.success)
    xs.append(x)
    ys.append(x)
    x_trains.append(x_train)
    x_tests.append(x_test)
    y_trains.append(y_train)
    y_tests.append(y_tests)

    means.append(f_mean_2)
    stds.append(f_std_2)
    lengthscales.append(gp.hyper_params[1])
    holeids.append(hole)
    sigma_ns.append(gp.sigma_n)

    pl.figure(figsize=(13, 5))
    pl.scatter(x_train, y_train, s=20, color ='red', label='Training points')
    pl.scatter(x_test, y_test,marker="x", s=20,  label='Test points')
    pl.plot(x_test2, f_mean_2, label='Mean prediction')
    pl.fill_between(x_test2.ravel(), (f_mean_2-2*f_std_2).ravel(), (f_mean_2+2*f_std_2).ravel(), \
                    facecolor='blue', interpolate=True, alpha=0.1, label='±2*sigma of prediction')
    #sns.lineplot(x=x_test.reshape(-1), y=f_mean_2.reshape(-1),color ='g')

    #sns.lineplot(x=x_test.reshape(-1), y=(f_mean_2-2*f_std_2).reshape(-1),color ='black')
    #sns.lineplot(x=x_test.reshape(-1), y=(f_mean_2+2*f_std_2).reshape(-1),color ='black')


    #sns.lineplot(x=x_test.reshape(-1), y=f_mean_2.reshape(-1),color ='g')
    pl.title('GP prediction of Cu concentrations vs depth ' + hole, fontsize=14)

    #pl.xlim([1500,1800])
    #pl.ylim([0,9])
    pl.xlabel('Depth (metres)')
    pl.ylabel('Cu concentration')
    pl.legend(loc='upper left')
    pl.grid()

### 4. Warped GP optimising lengthscale + variance

In [ ]:
np.random.seed(100) 
means = [] 
stds = []
lengthscales = []
holeids = []
xs = []
ys = []
x_trains = []
x_tests = []
y_trains = []
y_tests = []
convergeds=[]
alist = []
blist = []
clist = []
sigma_ns = []


#gp = GPR("Matern", x_train, y_train, [signal_variance,lengthscale], sigma_n, nu = 5/2)
I = 2
val_a = 0.4
val_b = 0.3
val_c = 0.1
a = np.full(I, val_a)
b = np.full(I, val_b)
c = np.full(I, val_c)

for hole in result['HOLEID'].unique():
    # subset assay data by domains
    adt_Cu = adt_Cu2.loc[adt_Cu2['HOLEID'] == hole]
    x = adt_Cu[['EAST_mid_tf','NORTH_mid_tf', 'RL_mid_tf']].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    y = adt_Cu[mineral].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    x = x.reshape(len(adt_Cu),3)
    x = x[:,2].reshape(-1,1)
    
    random_indices = np.random.permutation(x.shape[0])
    train_ratio = .7
    n_train = int(np.floor(x.shape[0]*train_ratio))
    n_test = x.shape[0]-n_train
    train_idx, test_idx = random_indices[:n_train], random_indices[n_train:]
    train_idx.sort()
    test_idx.sort()
    x_train, x_test = x[train_idx,:], x[test_idx,:]
    y_train, y_test = y[train_idx,:], y[test_idx,:]

    signal_variance = 1 #0.15
    lengthscale = 1000000 #50000
    sigma_n = 1 #0.1
    gpw = GaussianProcessRegressionLengthscaleVariance("Squared Exponential", x_train, y_train, [signal_variance,lengthscale], sigma_n,[a,b,c])
    gpw.fit()
    #[f_mean_2,f_std_2] = gpw.predict_original(x_test)
    x_test2 = np.linspace(x_test.min(), x_test.max(), 100).reshape(-1, 1)
    [f_mean_warp, f_std_warp] = gpw.predict(x_test2)
    convergeds.append(gpw.res.success)
    xs.append(x)
    ys.append(x)
    x_trains.append(x_train)
    x_tests.append(x_test)
    y_trains.append(y_train)
    y_tests.append(y_tests)

    means.append(f_mean_warp)
    stds.append(f_std_warp)

    sigma_ns.append(gpw.sigma_n)
    lengthscales.append(gpw.hyper_params[1])
    alist.append(gpw.gamma[0])
    blist.append(gpw.gamma[1])
    clist.append(gpw.gamma[2])


    holeids.append(hole)

    pl.figure(figsize=(13, 5))
    pl.scatter(x_train, gpw.hyp_tan(y_train), s=20, color ='red', label='Training points')
    pl.scatter(x_test, gpw.hyp_tan(y_test),marker="x", s=20, label='Test points')
    pl.plot(x_test2, f_mean_warp, label='Mean prediction')
    pl.fill_between(x_test2.ravel(), (f_mean_warp-2*f_std_warp).ravel(), (f_mean_warp+2*f_std_warp).ravel(), \
                    facecolor='blue', interpolate=True, alpha=0.1, label='±2*sigma of prediction')
    

    pl.title('Warped GP prediction of Cu concentrations vs depth ' + hole, fontsize=14)

    #pl.xlim([1500,1800])
    #pl.ylim([0,9])
    pl.xlabel('Depth (metres)')
    pl.ylabel('Cu concentration')
    pl.legend(loc='upper left')

    pl.grid()
    

### 5. GP using Cholesky decomposition, optimising lengthscale

In [ ]:
np.random.seed(100) 
means = [] 
stds = []
lengthscales = []
holeids = []
xs = []
ys = []
x_trains = []
x_tests = []
y_trains = []
y_tests = []
convergeds=[]
sigma_ns = []

for hole in result['HOLEID'].unique():
    # subset assay data by domains
    adt_Cu = adt_Cu2.loc[adt_Cu2['HOLEID'] == hole]
    x = adt_Cu[['EAST_mid_tf','NORTH_mid_tf', 'RL_mid_tf']].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    y = adt_Cu[mineral].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    x = x.reshape(len(adt_Cu),3)
    x = x[:,2].reshape(-1,1)

    random_indices = np.random.permutation(x.shape[0])
    train_ratio = .7
    n_train = int(np.floor(x.shape[0]*train_ratio))
    n_test = x.shape[0]-n_train
    train_idx, test_idx = random_indices[:n_train], random_indices[n_train:]
    train_idx.sort()
    test_idx.sort()
    x_train, x_test = x[train_idx,:], x[test_idx,:]
    y_train, y_test = y[train_idx,:], y[test_idx,:]

    signal_variance = 1 #0.15
    lengthscale = 1000000 #50000
    sigma_n = 1 #0.1
    #gp = GPR("Matern", x_train, y_train, [signal_variance,lengthscale], sigma_n, nu = 5/2)
    x_test2 = np.linspace(x_test.min(), x_test.max(), 100).reshape(-1, 1)

    gp = GaussianProcessRegressionLengthscaleCholesky("Squared Exponential", x_train, y_train, [signal_variance,lengthscale], sigma_n)
    gp.fit()
    [f_mean_2,f_std_2] = gp.predict(x_test2)

    xs.append(x)
    ys.append(x)
    x_trains.append(x_train)
    x_tests.append(x_test)
    y_trains.append(y_train)
    y_tests.append(y_tests)

    means.append(f_mean_2)
    stds.append(f_std_2)
    lengthscales.append(gp.hyper_params[1])
    holeids.append(hole)
    sigma_ns.append(gp.sigma_n)

    pl.figure(figsize=(13, 5))
    pl.scatter(x_train, y_train, s=20, color ='red')
    pl.scatter(x_test, y_test,marker="x", s=20)
    pl.plot(x_test2, f_mean_2)
    pl.fill_between(x_test2.ravel(), (f_mean_2-2*f_std_2).ravel(), (f_mean_2+2*f_std_2).ravel(), \
                    facecolor='blue', interpolate=True, alpha=0.1, label='±2*sigma of prediction')


    pl.title('GP prediction of Cu concentrations vs depth'+hole, fontsize=14)

    #pl.xlim([1500,1800])
    #pl.ylim([0,9])
    pl.xlabel('x [normalised]')
    pl.ylabel('y [normalised and zero mean]')
    pl.grid()  

### 6. GP using Cholesky decomposition, optimising lengthscale and variance

In [ ]:
np.random.seed(100) 
means = [] 
stds = []
lengthscales = []
holeids = []
xs = []
ys = []
x_trains = []
x_tests = []
y_trains = []
y_tests = []
convergeds=[]
sigma_ns = []

for hole in result['HOLEID'].unique():
    # subset assay data by domains
    adt_Cu = adt_Cu2.loc[adt_Cu2['HOLEID'] == hole]
    x = adt_Cu[['EAST_mid_tf','NORTH_mid_tf', 'RL_mid_tf']].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    y = adt_Cu[mineral].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    x = x.reshape(len(adt_Cu),3)
    x = x[:,2].reshape(-1,1)

    random_indices = np.random.permutation(x.shape[0])
    train_ratio = .7
    n_train = int(np.floor(x.shape[0]*train_ratio))
    n_test = x.shape[0]-n_train
    train_idx, test_idx = random_indices[:n_train], random_indices[n_train:]
    train_idx.sort()
    test_idx.sort()
    x_train, x_test = x[train_idx,:], x[test_idx,:]
    y_train, y_test = y[train_idx,:], y[test_idx,:]

    signal_variance = 1 #0.15
    lengthscale = 1000000 #50000
    sigma_n = 1 #0.1
    #gp = GPR("Matern", x_train, y_train, [signal_variance,lengthscale], sigma_n, nu = 5/2)
    x_test2 = np.linspace(x_test.min(), x_test.max(), 100).reshape(-1, 1)

    gp = GaussianProcessRegressionLengthscaleVarianceCholesky("Squared Exponential", x_train, y_train, [signal_variance,lengthscale], sigma_n)
    gp.fit()
    [f_mean_2,f_std_2] = gp.predict(x_test2)

    xs.append(x)
    ys.append(x)
    x_trains.append(x_train)
    x_tests.append(x_test)
    y_trains.append(y_train)
    y_tests.append(y_tests)

    means.append(f_mean_2)
    stds.append(f_std_2)
    lengthscales.append(gp.hyper_params[1])
    holeids.append(hole)
    sigma_ns.append(gp.sigma_n)

    pl.figure(figsize=(13, 5))
    pl.scatter(x_train, y_train, s=20, color ='red')
    pl.scatter(x_test, y_test,marker="x", s=20)
    pl.plot(x_test2, f_mean_2)
    pl.fill_between(x_test2.ravel(), (f_mean_2-2*f_std_2).ravel(), (f_mean_2+2*f_std_2).ravel(), \
                    facecolor='blue', interpolate=True, alpha=0.1, label='±2*sigma of prediction')


    pl.title('GP prediction of Cu concentrations vs depth'+hole, fontsize=14)

    #pl.xlim([1500,1800])
    #pl.ylim([0,9])
    pl.xlabel('x [normalised]')
    pl.ylabel('y [normalised and zero mean]')
    pl.grid()  

### 7. Warped GP using Cholesky decomposition, optimising lengthscale

In [ ]:
np.random.seed(100) 
means = [] 
stds = []
lengthscales = []
holeids = []
xs = []
ys = []
x_trains = []
x_tests = []
y_trains = []
y_tests = []
convergeds=[]
alist = []
blist = []
clist = []
sigma_ns = []


#gp = GPR("Matern", x_train, y_train, [signal_variance,lengthscale], sigma_n, nu = 5/2)
I = 1
val_a = 1
val_b = 1
val_c = 0.1
a = np.full(I, val_a)
b = np.full(I, val_b)
c = np.full(I, val_c)

for hole in result['HOLEID'].unique():
    # subset assay data by domains
    adt_Cu = adt_Cu2.loc[adt_Cu2['HOLEID'] == hole]
    x = adt_Cu[['EAST_mid_tf','NORTH_mid_tf', 'RL_mid_tf']].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    y = adt_Cu[mineral].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    x = x.reshape(len(adt_Cu),3)
    x = x[:,2].reshape(-1,1)
    
    random_indices = np.random.permutation(x.shape[0])
    train_ratio = .7
    n_train = int(np.floor(x.shape[0]*train_ratio))
    n_test = x.shape[0]-n_train
    train_idx, test_idx = random_indices[:n_train], random_indices[n_train:]
    train_idx.sort()
    test_idx.sort()
    x_train, x_test = x[train_idx,:], x[test_idx,:]
    y_train, y_test = y[train_idx,:], y[test_idx,:]

    signal_variance = 10 #1
    lengthscale = 200 # 1000000 
    sigma_n = 300 # 1
    gpw = GaussianProcessRegressionLengthscaleCholesky("Squared Exponential", x_train, y_train, [signal_variance,lengthscale], sigma_n,[a,b,c])
    gpw.fit()
    #[f_mean_2,f_std_2] = gpw.predict_original(x_test)
    x_test2 = np.linspace(x_test.min(), x_test.max(), 100).reshape(-1, 1)
    [f_mean_warp, f_std_warp] = gpw.predict(x_test2)
    convergeds.append(gpw.res.success)
    xs.append(x)
    ys.append(x)
    x_trains.append(x_train)
    x_tests.append(x_test)
    y_trains.append(y_train)
    y_tests.append(y_tests)

    means.append(f_mean_warp)
    stds.append(f_std_warp)

    sigma_ns.append(gpw.sigma_n)
    lengthscales.append(gpw.hyper_params[1])
    alist.append(gpw.gamma[0])
    blist.append(gpw.gamma[1])
    clist.append(gpw.gamma[2])


    holeids.append(hole)

    pl.figure(figsize=(13, 5))
    pl.scatter(x_train, gpw.hyp_tan(y_train), s=20, color ='red')
    pl.scatter(x_test, gpw.hyp_tan(y_test),marker="x", s=20)
    pl.plot(x_test2, f_mean_warp)
    pl.fill_between(x_test2.ravel(), (f_mean_warp-2*f_std_warp).ravel(), (f_mean_warp+2*f_std_warp).ravel(), \
                    facecolor='blue', interpolate=True, alpha=0.1, label='±2*sigma of prediction')
    

    pl.title('GP prediction of Cu concentrations vs depth'+hole, fontsize=14)

    #pl.xlim([1500,1800])
    #pl.ylim([0,9])
    pl.xlabel('x [normalised]')
    pl.ylabel('y [normalised and zero mean]')
    pl.grid()

### 8. Warped GP using Cholesky decomposition, optimising lengthscale and variance

In [ ]:
np.random.seed(100) 
means = [] 
stds = []
lengthscales = []
holeids = []
xs = []
ys = []
x_trains = []
x_tests = []
y_trains = []
y_tests = []
convergeds=[]
alist = []
blist = []
clist = []
sigma_ns = []


#gp = GPR("Matern", x_train, y_train, [signal_variance,lengthscale], sigma_n, nu = 5/2)
I = 1
val_a = 1
val_b = 1
val_c = 0.1
a = np.full(I, val_a)
b = np.full(I, val_b)
c = np.full(I, val_c)

for hole in result['HOLEID'].unique():
    # subset assay data by domains
    adt_Cu = adt_Cu2.loc[adt_Cu2['HOLEID'] == hole]
    x = adt_Cu[['EAST_mid_tf','NORTH_mid_tf', 'RL_mid_tf']].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    y = adt_Cu[mineral].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
    x = x.reshape(len(adt_Cu),3)
    x = x[:,2].reshape(-1,1)
    
    random_indices = np.random.permutation(x.shape[0])
    train_ratio = .7
    n_train = int(np.floor(x.shape[0]*train_ratio))
    n_test = x.shape[0]-n_train
    train_idx, test_idx = random_indices[:n_train], random_indices[n_train:]
    train_idx.sort()
    test_idx.sort()
    x_train, x_test = x[train_idx,:], x[test_idx,:]
    y_train, y_test = y[train_idx,:], y[test_idx,:]

    signal_variance = 10 #1
    lengthscale = 200 # 1000000 
    sigma_n = 0.1 # 1
    gpw = GaussianProcessRegressionLengthscaleVarianceCholesky("Squared Exponential", x_train, y_train, [signal_variance,lengthscale], sigma_n,[a,b,c])
    gpw.fit()
    #[f_mean_2,f_std_2] = gpw.predict_original(x_test)
    x_test2 = np.linspace(x_test.min(), x_test.max(), 100).reshape(-1, 1)
    [f_mean_warp, f_std_warp] = gpw.predict(x_test2)
    convergeds.append(gpw.res.success)
    xs.append(x)
    ys.append(x)
    x_trains.append(x_train)
    x_tests.append(x_test)
    y_trains.append(y_train)
    y_tests.append(y_tests)

    means.append(f_mean_warp)
    stds.append(f_std_warp)

    sigma_ns.append(gpw.sigma_n)
    lengthscales.append(gpw.hyper_params[1])
    alist.append(gpw.gamma[0])
    blist.append(gpw.gamma[1])
    clist.append(gpw.gamma[2])


    holeids.append(hole)

    pl.figure(figsize=(13, 5))
    pl.scatter(x_train, gpw.hyp_tan(y_train), s=20, color ='red')
    pl.scatter(x_test, gpw.hyp_tan(y_test),marker="x", s=20)
    pl.plot(x_test2, f_mean_warp)
    pl.fill_between(x_test2.ravel(), (f_mean_warp-2*f_std_warp).ravel(), (f_mean_warp+2*f_std_warp).ravel(), \
                    facecolor='blue', interpolate=True, alpha=0.1, label='±2*sigma of prediction')
    

    pl.title('GP prediction of Cu concentrations vs depth'+hole, fontsize=14)

    #pl.xlim([1500,1800])
    #pl.ylim([0,9])
    pl.xlabel('x [normalised]')
    pl.ylabel('y [normalised and zero mean]')
    pl.grid()

In [ ]:
a = gpw.K + np.eye(gpw.K.shape[0]) * (gpw.sigma_n ** 2)
a

In [ ]:
gpw.K + np.eye(gpw.K.shape[0]) * (gpw.sigma_n )

In [ ]:
gpw.sigma_n